In [1]:
'''PROGRAM LOGIC
-Merge into one dataframe
-Stats analysis
'''

'PROGRAM LOGIC\n-Merge into one dataframe\n-Stats analysis\n'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu
from statsmodels.stats.proportion import proportions_ztest

In [23]:
df_rc = pd.read_csv('pd_patients_redcap.csv')

In [24]:
df_g = pd.read_csv('pd_patients_google.csv')

In [27]:
display(df_rc.head(0))
display(df_g.head(0))

print(len(df_rc), len(df_g))

frames = [df_rc, df_g]

df_combo = pd.concat(frames, ignore_index=True)

display(df_combo.head())
print(len(df_combo))


,Record ID,What is your age?,What is your current employment status?,"If you are retired, what was your last job?",What is your job/profession?,What is your gender?&nbsp;,"Are you of Hispanic, Latino/a, or Spanish origin?",What is your race? (Check all that apply) (choice=White),What is your race? (Check all that apply) (choice=Black or African American),What is your race? (Check all that apply) (choice=American Indian or Alaska Native),...,How long have you been diagnosed with Parkinson's Disease?,How many people do you personally know with Parkinson's Disease?,How often do you make video calls or post videos online?,I would like to have the option to remove tremors from videos of myself.,I would like for viewers to have the option to remove tremors from a video of myself.,My tremors affect my level of confidence.,I feel like my tremors cause others to perceive me as less competent.,Would you like to be contacted in the future for a follow-up interview or other research projects?,What is your email? (optional),Complete?


,Timestamp,Username,What is your age?,What is your current employment status?,"If you are retired, what was your last job?",What is your job/profession?,What is your gender?,"Are you of Hispanic, Latino/a, or Spanish origin?",What is your race? (Check all that apply),What the highest level of education you have obtained?,...,How many people do you personally know with Parkinson’s Disease?,How often do you make video calls or post videos online?,I would like to have the option to remove tremors from videos of myself.,I would like for viewers to have the option to remove tremors from a video of myself.,My tremors affect my level of confidence.,I feel like my tremors cause others to perceive me as less competent.,What is your email? (optional),Would you like to be contacted in the future for a follow-up interview or other research projects?,Unnamed: 21,Unnamed: 22


101 76


,Record ID,What is your age?,What is your current employment status?,"If you are retired, what was your last job?",What is your job/profession?,What is your gender?&nbsp;,"Are you of Hispanic, Latino/a, or Spanish origin?",What is your race? (Check all that apply) (choice=White),What is your race? (Check all that apply) (choice=Black or African American),What is your race? (Check all that apply) (choice=American Indian or Alaska Native),...,Complete?,Timestamp,Username,What is your gender?,What is your race? (Check all that apply),What the highest level of education you have obtained?,Are you part of a Parkinson's Disease support group?,How many people do you personally know with Parkinson’s Disease?,Unnamed: 21,Unnamed: 22
0,1.0,45-54 years old,Employed,NaN,accountant,Female,"Not of Hispanic, Latino/a, or Spanish origin",Checked,Unchecked,Unchecked,...,Complete,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,75-84 years old,Retired,Software designer,NaN,Male,"Not of Hispanic, Latino/a, or Spanish origin",Checked,Unchecked,Unchecked,...,Complete,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,45-54 years old,Other,NaN,Disabled via Parkinson's,Male,"Not of Hispanic, Latino/a, or Spanish origin",Checked,Unchecked,Unchecked,...,Complete,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,65-74 years old,Retired,Buyer,NaN,Male,"Not of Hispanic, Latino/a, or Spanish origin",Checked,Unchecked,Unchecked,...,Complete,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,55-64 years old,Retired,Aerospace engineering,NaN,Female,"Not of Hispanic, Latino/a, or Spanish origin",Checked,Unchecked,Unchecked,...,Incomplete,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


177


In [28]:
#Self-contained cell for converting data into numerical format for strongly agree questions
str2int_dict = {'Strongly Agree':5.0, 'Agree':4.0, 'Undecided': 3.0,'Disagree':2.0, 'Strongly Disagree':1.0}
def str2int(st):
    if ((type(st) == str) and (st.endswith('agree') or st.endswith('Agree') or st.endswith('decided'))):
        return str2int_dict[st]
    else:
        return st



In [30]:
for col in df_combo.columns:
    df_combo[col] = df_combo[col].apply(str2int)


In [31]:
display(df_combo.head(10))

,Record ID,What is your age?,What is your current employment status?,"If you are retired, what was your last job?",What is your job/profession?,What is your gender?&nbsp;,"Are you of Hispanic, Latino/a, or Spanish origin?",What is your race? (Check all that apply) (choice=White),What is your race? (Check all that apply) (choice=Black or African American),What is your race? (Check all that apply) (choice=American Indian or Alaska Native),...,Complete?,Timestamp,Username,What is your gender?,What is your race? (Check all that apply),What the highest level of education you have obtained?,Are you part of a Parkinson's Disease support group?,How many people do you personally know with Parkinson’s Disease?,Unnamed: 21,Unnamed: 22
0,1.0,45-54 years old,Employed,NaN,accountant,Female,"Not of Hispanic, Latino/a, or Spanish origin",Checked,Unchecked,Unchecked,...,Complete,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,75-84 years old,Retired,Software designer,NaN,Male,"Not of Hispanic, Latino/a, or Spanish origin",Checked,Unchecked,Unchecked,...,Complete,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,45-54 years old,Other,NaN,Disabled via Parkinson's,Male,"Not of Hispanic, Latino/a, or Spanish origin",Checked,Unchecked,Unchecked,...,Complete,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,65-74 years old,Retired,Buyer,NaN,Male,"Not of Hispanic, Latino/a, or Spanish origin",Checked,Unchecked,Unchecked,...,Complete,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,55-64 years old,Retired,Aerospace engineering,NaN,Female,"Not of Hispanic, Latino/a, or Spanish origin",Checked,Unchecked,Unchecked,...,Incomplete,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6.0,55-64 years old,Retired,Aerospace engineering,NaN,Male,"Not of Hispanic, Latino/a, or Spanish origin",Checked,Unchecked,Unchecked,...,Complete,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,8.0,55-64 years old,Retired,Registered Nurse,NaN,Female,"Not of Hispanic, Latino/a, or Spanish origin",Checked,Unchecked,Unchecked,...,Complete,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,9.0,65-74 years old,Retired,EDUCATOR,NaN,Female,"Not of Hispanic, Latino/a, or Spanish origin",Checked,Unchecked,Unchecked,...,Complete,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,10.0,75-84 years old,Retired,Ship pilot,NaN,Male,"Not of Hispanic, Latino/a, or Spanish origin",Checked,Unchecked,Unchecked,...,Complete,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,11.0,55-64 years old,Retired,Comcast Cable,NaN,Male,"Not of Hispanic, Latino/a, or Spanish origin",Checked,Unchecked,Unchecked,...,Complete,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
# Create an array of questions to analyze
q0 = "I would like to have the option to remove tremors from videos of myself."
q1 = "I would like for viewers to have the option to remove tremors from a video of myself."
q2 = "My tremors affect my level of confidence. " 
q3 = "I feel like my tremors cause others to perceive me as less competent. "

q_list = [q0, q1, q2, q3]

# There was a space in the google forms for q2 and q3

In [33]:
# This is to generate a table of distributions (counts) for each answer for each question
# 'Strongly Agree':5, 'Agree':4, 'Undecided': 3,'Disagree':2, 'Strongly Disagree':1
# str2int_dict2 = {'Always':5, 'Often':4, 'Sometimes': 3,'Rarely':2, 'Never':1}
df_counts = pd.DataFrame()
for q in q_list:
    col_name = q
    df_counts[col_name] = df_combo[col_name].value_counts() # applies function for every row in specified column
display(df_counts)

,I would like to have the option to remove tremors from videos of myself.,I would like for viewers to have the option to remove tremors from a video of myself.,My tremors affect my level of confidence.,I feel like my tremors cause others to perceive me as less competent.
3.0,63,59,30,30
4.0,57,61,55,71
2.0,22,16,44,31
5.0,19,18,16,19
1.0,9,16,20,14


In [35]:
# here we create a dataframe containing only disagree and agree sums
# Keep in mind there is a frequency question... close enough ?
df_agree = df_counts.loc[[4.0, 5.0],:].sum()
df_disagree = df_counts.loc[[1.0,2.0],:].sum()
df_neutral = df_counts.loc[[3.0],:].sum()
df_AD = pd.concat([df_agree, df_disagree], axis=1)
df_AD.columns = ['Agree','Disagree']
print("Count for those who AGREED to the questions: ")
display(df_agree)
print("Count for those who DISAGREED to the questions: ")
display(df_disagree)
display(df_AD)

Count for those who AGREED to the questions: 


I would like to have the option to remove tremors from videos of myself.                 76
I would like for viewers to have the option to remove tremors from a video of myself.    79
My tremors affect my level of confidence.                                                71
I feel like my tremors cause others to perceive me as less competent.                    90
dtype: int64

Count for those who DISAGREED to the questions: 


I would like to have the option to remove tremors from videos of myself.                 31
I would like for viewers to have the option to remove tremors from a video of myself.    32
My tremors affect my level of confidence.                                                64
I feel like my tremors cause others to perceive me as less competent.                    45
dtype: int64

,Agree,Disagree
I would like to have the option to remove tremors from videos of myself.,76,31
I would like for viewers to have the option to remove tremors from a video of myself.,79,32
My tremors affect my level of confidence.,71,64
I feel like my tremors cause others to perceive me as less competent.,90,45


In [22]:
# testing for population differences on all questions
# statsmodels.stats.proportion.proportions_ztest(count, nobs, value=None, 
#                   alternative='two-sided', prop_var=False)
# Returns statistic and p-value
# Null Hypothesis = There is no difference in public opinion. 
    # E.g., Number who agree is equal to the number who disagree

for q in q_list:
    print(q)
    row_ID = q
    num_agree = df_AD.loc[row_ID,'Agree']
    num_disagree = df_AD.loc[row_ID,'Disagree']
    t, p = proportions_ztest(num_agree, (num_agree+num_disagree), value=.5) # Assume no difference 
    print("T: ", t, "P: ", p)
    print("\n==========================\n")

I would like to have the option to remove tremors from videos of myself.
T:  4.79497960077169 P:  1.6269138550665999e-06


I would like for viewers to have the option to remove tremors from a video of myself.
T:  4.924258200898274 P:  8.468096480949749e-07


My tremors affect my level of confidence. 
T:  0.6032756097072094 P:  0.5463253557295227


I feel like my tremors cause others to perceive me as less competent. 
T:  4.107919181288745 P:  3.992397476901354e-05


